In [15]:
import pandas as pd
from IPython.display import display

# Load the Excel file
file_path = r"C:\Users\iamve.LEGION\Downloads\large_pharma_sales_data_english.xlsx"

xls = pd.ExcelFile(file_path)

# Load all sheets into a dictionary of DataFrames
dataframes = {sheet: xls.parse(sheet) for sheet in xls.sheet_names}

# Extract the needed dataframes
sales_df = dataframes['Sales']
drugs_df = dataframes['Drugs']

# Strip extra spaces in key columns
sales_df['Drug ID'] = sales_df['Drug ID'].astype(str).str.strip()
drugs_df['Drug ID'] = drugs_df['Drug ID'].astype(str).str.strip()

# Merge Drug Names into Sales
sales_df = pd.merge(sales_df, drugs_df[['Drug ID', 'Drug Name']], on='Drug ID', how='left')

# Display Original Data
for sheet, df in dataframes.items():
    print(f"\nOriginal Data - Sheet: {sheet}")
    df.columns = df.columns.str.strip()  # Remove leading/trailing spaces in column names
    display(df.head())
    print("-" * 40)


Original Data - Sheet: Sales


,Sale ID,Invoice ID,Drug ID,Quantity Sold,Price per Unit,Total Price,City,Manufacturer,Supplier ID
0,752658,13242,D101,47,89.97,4228.59,Luxor,Sanofi,S203
1,810107,17204,D101,35,89.97,3148.95,Mansoura,Sanofi,S200
2,841475,17733,D105,74,74.38,5504.12,Cairo,Pfizer,S202
3,574213,16076,D105,82,74.38,6099.16,Luxor,Pfizer,S202
4,631870,15705,D102,58,77.59,4500.22,Cairo,Pfizer,S200


----------------------------------------

Original Data - Sheet: Drugs


,Drug ID,Drug Name,Manufacturer,Category,Base Price
0,D100,Paracetamol,Pfizer,Pain Relievers,48.72
1,D101,Ibuprofen,Sanofi,Anti-Inflammatory,89.97
2,D102,Amoxicillin,Pfizer,Antibiotics,77.59
3,D103,Ciprofloxacin,Sanofi,Antibiotics,110.02
4,D104,Metformin,Pfizer,Diabetes Treatment,64.12


----------------------------------------

Original Data - Sheet: Manufacturers


...

----------------------------------------

Original Data - Sheet: Suppliers


,Supplier ID,Supplier Name,Country,Year Established
0,S200,MediSupply,USA,2013
1,S201,PharmaGlobal,Germany,2000
2,S202,HealthLine,India,2009
3,S203,MedixCorp,Germany,1991
4,S204,BioPharm,India,1983


----------------------------------------

Original Data - Sheet: Invoices


,Invoice ID,Customer ID,Invoice Date,Total Amount
0,13242,C1186,2024-01-22,4228.59
1,17204,C0574,2024-02-27,3148.95
2,17733,C0992,2024-02-10,5504.12
3,16076,C0643,2024-02-04,6099.16
4,15705,C0903,2024-01-18,4500.22


----------------------------------------


In [16]:
# Best-Selling Product by Name
if {'Drug Name', 'Quantity Sold'}.issubset(sales_df.columns):
    best_selling_product = sales_df.groupby('Drug Name')['Quantity Sold'].sum().idxmax()
    print(f"\nBest-Selling Product: {best_selling_product}")
else:
    print("\nBest-Selling Product: Data unavailable")

# Sales Trend over Months and Years
if 'Invoice ID' in sales_df.columns: 
    sales_df['Year'] = sales_df['Invoice ID'].astype(str).str[:4].astype(int)
    sales_df['Month'] = sales_df['Invoice ID'].astype(str).str[4:6].astype(int)
    sales_df = sales_df[(sales_df['Year'] >= 1950) & (sales_df['Year'] <= 2030)]  # Filter years between 1950 and 2030
    sales_df = sales_df[(sales_df['Month'] >= 1) & (sales_df['Month'] <= 12)]  # Ensure valid months
    monthly_sales = sales_df.groupby(['Year', 'Month'])['Total Price'].sum()  # Group by Year and Month, then sum Total Price to analyze monthly sales trends
    print("\nSales Trend over Months and Years:")
    display(monthly_sales.to_frame(name='Monthly Sales'))
else:
    print("\nSales Trend over Months and Years: Data unavailable")


Best-Selling Product: Ciprofloxacin

Sales Trend over Months and Years:


Monthly Sales
Year Month               
1950 3            3800.16
     4            4488.40
     5             128.24
     9           25589.59
1951 2            2582.16
...                   ...
1998 1           10636.76
     3           19420.64
     7           16185.38
1999 7            5043.35
     9            7737.42

[164 rows x 1 columns]

In [17]:
# 3. Customer and Geographic Analysis
print("\nOriginal Customer and Geographic Data:")
if {'City', 'Total Price'}.issubset(sales_df.columns):
    display(sales_df[['City', 'Total Price']].head())
    city_sales = sales_df.groupby('City')['Total Price'].sum().sort_values(ascending=False)
    display(city_sales.to_frame(name='Total Sales by City'))
else:
    print("Customer and geographic analysis: Data unavailable")


Original Customer and Geographic Data:


,City,Total Price
20,Tanta,3718.96
28,Asyut,4070.74
50,Suez,3788.72
60,Asyut,539.82
90,Giza,3556.56


,Total Sales by City
City,
Cairo,163605.55
Luxor,160242.23
Alexandria,155046.72
Suez,154992.55
Tanta,144670.68
Mansoura,122767.68
Asyut,88641.79
Giza,65117.62


In [18]:
# 4. Product and Pricing Analysis (Using Drug Name)
print("\nOriginal Product Data:")
if {'Drug Name', 'Price per Unit', 'Quantity Sold', 'Total Price'}.issubset(sales_df.columns):  
    display(sales_df[['Drug Name', 'Price per Unit', 'Quantity Sold']].head())
    
    # Average price per product (by name)
    average_price_per_product = sales_df.groupby('Drug Name')['Price per Unit'].mean()
    
    # Correlation between price and quantity sold
    price_quantity_correlation = sales_df[['Price per Unit', 'Quantity Sold']].corr()
    
    display(average_price_per_product.to_frame(name='Average Price per Product'))
    display(price_quantity_correlation)
else:
    print("Product and pricing analysis: Data unavailable")


Original Product Data:


,Drug Name,Price per Unit,Quantity Sold
20,Metformin,64.12,58
28,Ciprofloxacin,110.02,37
50,Losartan,145.72,26
60,Ibuprofen,89.97,6
90,Paracetamol,48.72,73


,Average Price per Product
Drug Name,
Amoxicillin,77.59
Aspirin,74.38
Atorvastatin,143.74
Ciprofloxacin,110.02
Ibuprofen,89.97
Losartan,145.72
Metformin,64.12
Paracetamol,48.72


,Price per Unit,Quantity Sold
Price per Unit,1.000000,0.045663
Quantity Sold,0.045663,1.000000
